In [1]:
from typing import List, Optional, Generator
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

#### Utility to generate random BST

In [4]:
def generateBST(N: int, min_: int, max_: int) -> BinaryTreeNode|None:
    def insert(curr: BinaryTreeNode|None, n: int) -> BinaryTreeNode:
        if not curr:
            curr = BinaryTreeNode(n)
        elif curr.val < n:
            curr.right = insert(curr.right, n)
        else:
            curr.left = insert(curr.left, n)

        return curr

    assert N <= max_ - min_, "Number of available samples must be >= N"
    root: BinaryTreeNode|None = None
    for n in np.random.choice(np.arange(min_, max_), size=N, replace=False):
        root = insert(root, n)

    return root

Best time to buy and sell stocks - I

In [5]:
def stocks1(prices: list[int]) -> int:
    min_, max_profit = prices[0], 0
    for price in prices:
        max_profit = max(max_profit, price - min_)
        min_ = min(min_, price)

    return max_profit

# Testing the solution
assert stocks1([7,1,5,3,6,4]) == 5
assert stocks1([7,6,4,3,1]) == 0

Weekly Contest 398: 19th May 2024

In [6]:
# Q1
def isArraySpecial(nums: list[int]) -> bool:
    N = len(nums)
    for i in range(N - 1):
        i1, i2 = nums[i] % 2, nums[i + 1] % 2
        if (i1 == 0 and i2 == 0) or (i1 == 1 and i2 == 1):
            return False
    else:
        return True

# Testing the solution
assert isArraySpecial([4,3,1,6]) == False
assert isArraySpecial([2,1,4]) == True
assert isArraySpecial([1]) == True

In [7]:
# Q2
def isArraySpecial2(nums: list[int], queries: list[list[int]]) -> list[bool]:
    N = len(nums)
    parity: list[int] = list(map(lambda x: x % 2, nums))

    @functools.cache
    def backtrack(start: int, end: int) -> bool:
        if start == end:
            return True
        else:
            mid = (start + end) // 2
            isSpecial =  parity[mid] != parity[mid + 1] and backtrack(start, mid) and backtrack(mid + 1, end)
            return isSpecial

    result: list[bool] = []
    for i, j in queries:
        result.append(backtrack(i, j))

    return result

# Testing the solution
assert isArraySpecial2([4,3,1,6], [[0,2],[2,3]]) == [False, True]
assert isArraySpecial2([3,4,1,2,6], [[0,4]]) == [False]

In [8]:
# Q3
def sumDigitDifferencesBrute(nums: list[int]) -> int:
    length = len(str(nums[0]))
    count = 0
    for n1, n2 in itertools.combinations(nums, r=2):
        n1_str, n2_str = str(n1), str(n2)
        for i in range(length):
            if n1_str[i] != n2_str[i]:
                count += 1

    return count

def sumDigitDifferences(nums: list[int]) -> int:
    def count(arr: list[int]) -> int:
        N = len(arr)
        temp = 0
        for i in range(N - 1):
            temp += arr[i] * sum(arr[i + 1:])

        return temp

    length = len(str(nums[0]))
    digits_dict: dict[int, dict[str, int]] = dict()
    for n in nums:
        n_str = str(n)
        for i in range(length):
            positions = digits_dict.get(i, dict())
            positions[n_str[i]] = positions.get(n_str[i], 0) + 1
            digits_dict[i] = positions

    result: int = 0
    for i in range(length):
        result += count(list(digits_dict[i].values()))

    return result

# Testing the solution
for i in range(100):
    temp = list(np.random.randint(10, 100, 50))
    assert sumDigitDifferencesBrute(temp) == sumDigitDifferences(temp)

Sum of XOR totals
Easy LC: https://leetcode.com/problems/sum-of-all-subset-xor-totals

In [9]:
# https://leetcode.com/problems/sum-of-all-subset-xor-totals/submissions/1263288868
def subsetXORSumMemo(nums: list[int]) -> int:
    @functools.cache
    def backtrack(i: int, curr: int) -> int:
        if i == N:
            return curr
        else:
            return backtrack(i + 1, curr ^ nums[i]) + backtrack(i + 1, curr)

    N = len(nums)
    return backtrack(0, 0)

# Testing the solution
assert subsetXORSumMemo([1,3]) == 6
assert subsetXORSumMemo([5,1,6]) == 28

Largest Rectangle in a histogram

In [10]:
# https://leetcode.com/problems/largest-rectangle-in-histogram/submissions/1266278725
def largestRectangleArea(heights: list[int]) -> int:
    """
    Count the number of rectangles to the right that are greater than or equal to self
    Count the number of rectangles to the left that are greater than or equal to self
    """

    N = len(heights)

    right_dp: list[int] = [N - i for i in range(N)]
    left_dp: list[int] = [i + 1 for i in range(N)]
    stack: list[tuple[int, int]] = []

    for i in range(N):
        curr: tuple[int, int] = (heights[i], i)
        while stack and stack[-1][0] > curr[0]:
            prev = stack.pop()
            right_dp[prev[1]] = curr[1] - prev[1]
        stack.append(curr)

    stack.clear()

    for i in range(N - 1, -1, -1):
        curr = (heights[i], i)
        while stack and stack[-1][0] > curr[0]:
            prev = stack.pop()
            left_dp[prev[1]] = prev[1] - curr[1]
        stack.append(curr)

    max_ = 0
    for i in range(N):
        max_ = max(max_, (left_dp[i] + right_dp[i] - 1) * heights[i])

    return max_

# Testing the solution
assert largestRectangleArea([2,1,5,6,2,3]) == 10
assert largestRectangleArea([2,4]) == 4
assert largestRectangleArea([2,1,2]) == 3

Container Queries - CN Weekly - 3

In [11]:
def containerQueries(N: int, S: str, query_len: int, queries: list[list[int]]) -> list[int]:
    positions: dict[str, collections.deque[int]] = dict()
    for i in range(N):
        indices = positions.get(S[i], collections.deque())
        indices.append(i)
        positions[S[i]] = indices

    end_pos = 0
    for k in positions:
        end_pos = max(end_pos, positions[k].popleft())

    dp: list[int] = [end_pos]
    for i in range(1, N):
        prev = S[i - 1]
        next_pos = max(dp[-1], positions[prev].popleft() if positions[prev] else N)
        dp.append(next_pos)

    result: list[int] = []
    for i, j in queries:
        if j >= dp[i]:
            result.append(1)
        else:
            result.append(0)

    return result

# Testing the solution
assert containerQueries(8, "yukppuyk", 2, [[0,4],[5,7]]) == [1, 0]

LC Biweekly 131: 25th May 2024

In [12]:
# Q1
def duplicateNumbersXOR(nums: list[int]) -> int:
    set_: set[int] = set()
    result: int = 0
    for n in nums:
        if n not in set_:
            set_.add(n)
        else:
            result = result ^ n

    return result

# Testing the solution
assert duplicateNumbersXOR([1,2,1,3]) == 1
assert duplicateNumbersXOR([1,2,3]) == 0
assert duplicateNumbersXOR([1,2,2,1]) == 3

In [13]:
# Q2
def occurrencesOfElement(nums: list[int], queries: list[int], x: int) -> list[int]:
    N = len(nums)
    occurance: int = 0
    indices: dict[int, int] = dict()
    for i in range(N):
        if nums[i] == x:
            indices[occurance] = i
            occurance += 1

    result: list[int] = []
    for q in queries:
        result.append(indices.get(q - 1, -1))

    return result

# Testing the solution
assert occurrencesOfElement([1,1,1], [1,2,3,4], 1) == [0,1,2,-1]
assert occurrencesOfElement([1,3,1,7], [1,3,2,4], 1) == [0,-1,2,-1]
assert occurrencesOfElement([1,2,3], [10], 5) == [-1]

In [14]:
# Q3
def queryResults(limit: int, queries: list[list[int]]) -> list[int]:
    color_map: dict[int, set[int]] = dict()
    ball_map: dict[int, int] = dict()

    result: list[int] = []
    for ball, color in queries:
        old_color = ball_map.get(ball, 0)
        if old_color > 0:
            color_map[old_color].remove(ball)
            if len(color_map[old_color]) == 0:
                del color_map[old_color]

        balls = color_map.get(color, set())
        balls.add(ball)
        color_map[color] = balls
        ball_map[ball] = color

        result.append(len(color_map))

    return result

# Testing the solution
assert queryResults(4, [[0,1],[1,2],[2,2],[3,4],[4,5]]) == [1,2,2,3,4]
assert queryResults(4, [[1,4],[2,5],[1,3],[3,4]]) == [1,2,2,3]
assert queryResults(1, [[0,10],[1,10]]) == [1,1]

In [15]:
# Q4: TLE :(
import bisect

def getResultsBrute(queries: list[list[int]]) -> list[bool]:
    obstacles: list[int] = [0]
    result: list[bool] = []

    for q in queries:
        if len(q) == 2:
            bisect.insort(obstacles, q[1])
        else:
            x, sz = q[1], q[2]
            idx = bisect.bisect_left(obstacles, x)
            max_size = x - obstacles[idx - 1]
            for i in range(1, idx):
                max_size = max(max_size, obstacles[i] - obstacles[i - 1])

            result.append(max_size >= sz)

    return result

# Testing the solution
assert getResultsBrute([[1,2],[2,3,3],[2,3,1],[2,2,2]]) == [False, True, True]
assert getResultsBrute([[1,7],[2,7,6],[1,2],[2,7,5],[2,7,6]]) == [True,True,False]

CN Weekly Contest

In [16]:
# Q2 - Easy
def minimumLength(s1: str, s2: str) -> int:
    N, M = len(s1), len(s2)

    if s1 in s2:
        return M

    elif s2 in s1:
        return N

    else:
        # Store the indices where s1[0] and s1[-1] are found
        left: list[int] = []
        right: list[int] = []
        for i in range(M):
            if s2[i] == s1[0]:
                right.append(i)
            if s2[i] == s1[-1]:
                left.append(i)

        # Ans is always possible
        min_ = N + M

        # Check right concatentation
        for i in right:
            overlap_length = M - i
            if s2[i:] == s1[:overlap_length]:
                min_ = min(min_, N + M - overlap_length)

        # Check left concatentation
        for i in left:
            overlap_length = i + 1
            if s2[:i + 1] == s1[-overlap_length:]:
                min_ = min(min_, N + M - overlap_length)

        return min_

# Testing the solution
assert minimumLength("abcabc", "aaabc") == 8
assert minimumLength("bibia", "aibia") == 9
assert minimumLength("aabcabcd", "abc") == 8

LC Medium: Count number of primes
https://leetcode.com/problems/count-primes/

In [17]:
# https://leetcode.com/problems/count-primes/submissions/1275257285
def countPrimes(N: int) -> int:

    if N == 0:
        return 0

    soe: list[bool] = [True] * (N + 1)
    soe[0] = soe[1] = False
    i = 2
    while i * i <= N:
        if soe[i]:
            """
            j = i
            while i * j <= N:
                soe[i * j] = False
                j += 1
            """
            # List slicing is much faster thanks to the C impl backend
            soe[i*i:N+1:i] = [False] * len(soe[i*i:N+1:i])
        i += 1

    return sum(soe)

# Testing the solution
assert countPrimes(10) == 4
assert countPrimes(0) == countPrimes(1) == 0
assert countPrimes(100) == 25

Heights checker - Easy
https://leetcode.com/problems/height-checker/submissions/1283296449/

In [ ]:
def heightChecker(heights: list[int]) -> int:
    """
    Implement count sort
    Time: O(N), Space: O(N)

    1. Bucketize and count the heights
    2. Kind of like a two pointer approach, iterate through both. Bucketized dict is iterate in order of natural numbers O(100 * individual freq) ~ O(N)
    3. This solution is O(N) only because we have a natural limit to max height - 100
    """
    freq = collections.Counter(heights)

    result = currHeight = 0
    for height in heights:

        while currHeight not in freq or freq[currHeight] == 0: 
            currHeight += 1

        if currHeight != height:
            result += 1

        # Whether matching or not matching, one of student's
        # heights were compared, so continue travesing
        freq[currHeight] -= 1

    return result

# Testing the solution
assert heightChecker([1,1,4,2,1,3]) == 3
assert heightChecker([5,1,2,3,4]) == 5
assert heightChecker([1,2,3,4,5]) == 0

LC Virtual - 1: Warmup contest
https://leetcode.com/contest/warm-up-contest

In [ ]:
# Q1: Lexicographical numbers
def lexicalOrderRecursive(n: int) -> list[int]:
    result: list[int] = []
    def backtrack(curr: int) -> None:
        result.append(curr)
        if curr * 10 <= n:
            backtrack(curr * 10)
        if (curr % 10) < 9 and curr < n:
            backtrack(curr + 1)

    backtrack(1)
    return result

# Testing the solution
lexicalOrderRecursive(25)

In [ ]:
def lexicalOrder(n: int) -> list[int]:
    result: list[int] = []
    stack: list[int] = [1]
    while stack:
        curr = stack.pop()
        result.append(curr)
        if (curr % 10) < 9 and curr < n:
            stack.append(curr + 1)
        if curr * 10 <= n:
            stack.append(curr * 10)

    return result

# Testing the solution
lexicalOrder(25)

In [ ]:
# Q2: First unique character in a string
def firstUniqChar(s: str) -> int:
    """
    Iterate from right to left. Keep track of elements already visited.
    If new char is encountered update result.
    Else (char already exists), ensure that result prev found
    """
    N = len(s)
    freq = collections.Counter(s)
    for i in range(N):
        if freq[s[i]] == 1:
            return i
    return -1

# Testing the solution
assert firstUniqChar("leetcode") == 0
assert firstUniqChar("loveleetcode") == 2
assert firstUniqChar("aabb") == -1
assert firstUniqChar("aab") == 2

In [ ]:
# Q3: Longest Absolute File Path
def lengthLongestPath(abstracted: str) -> int:
    """
    Iterate through path character by character.
    "\n" demarcates the structure.
    "\t" tells us about the depth

    If curr_depth is > prev_depth, nest inside previous.
    If curr_depth <= prev_depth, remove previously added levels and add curr. Levels to remove: prev_depth - curr_depth + 1
    """

    path: list[str] = []
    N, i = len(abstracted), 0
    max_length, depth = 0, 0
    is_file = False
    while i < N:
        if abstracted[i] == "\n":
            # Check if current absolute path is a file, if yes update max_length
            if is_file:
                max_length = max(max_length, len(path))

            i, prev_depth, depth = i + 1, depth, 0
            while abstracted[i] == "\t":
                i, depth = i + 1, depth + 1

            # Remove nested structure if curr_depth <= prev_depth
            if depth <= prev_depth:
                extra_depth = prev_depth - depth + 1
                while path and extra_depth:
                    removed = path.pop()
                    if removed == "/":
                        extra_depth -= 1

            # Reset values for upcoming cycle
            if depth > 0:
                path.append("/")
            is_file = False

        else:
            if abstracted[i] == '.':
                is_file = True
            path.append(abstracted[i])
            i += 1

    return max(max_length, len(path) if is_file else 0)

# Testing the solution
assert lengthLongestPath("dir\n\tsubdir1\n\t\tfile1.ext\n\t\tsubsubdir1\n\tsubdir2\n\t\tsubsubdir2\n\t\t\tfile2.ext") == 32
assert lengthLongestPath("dir\n\tsubdir1\n\tsubdir2\n\t\tfile.ext") == 20
assert lengthLongestPath("file1.txt\nfile2.txt\nlongfile.txt") == 12
assert lengthLongestPath("dir1\ndir2\n\tsubdir1\n\tsubdir2\n\t\tfile.ext") == 21
assert lengthLongestPath("dir0\n\tabcdefghijklmnopqrstuvw.file") == 33